In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import re
#import time

In [2]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
open_eye_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
left_eye_cascade = cv.CascadeClassifier('haarcascade_lefteye_2splits.xml')
right_eye_cascade = cv.CascadeClassifier('haarcascade_righteye_2splits.xml')

In [3]:
eye_status_clf = tf.keras.models.load_model('Lenet-5-CNN-20Epcoh-96Ac.h5')

In [4]:
model_input_shape = (64,64,3)
def predict(image, model=eye_status_clf):
 

    if image.shape != model_input_shape:
        image = cv.resize(image, (model_input_shape[0], model_input_shape[1]))
    
    
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    pred = model.predict(image).argmax(axis=1)
   
    if pred ==0: 
        status = 'Close'
    else:
        status = 'Open'
    
    return str(pred[0])

In [10]:
# Capture WebCam Vedio
video = cv.VideoCapture(0)
eye_status = '' #Open
frameCount = 0
status = ''
while(frameCount<20):
    ret, frame = video.read()
    
    grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    face = face_cascade.detectMultiScale(grayImage)
    
    for (x,y,w,h) in face:
        #print('Face Detected....')
        cv.rectangle(frame,(x,y),(x+w,y+w),(255,0,0),2)
        face_gray = grayImage[y:y+h, x:x+w]
        face_color = frame[y:y+h, x:x+w]
        
        '''plt.imshow(face_color)
        plt.title('Face')
        plt.show()'''
        eye = open_eye_cascade.detectMultiScale(face_gray)
        if (len(eye) == 2):   # Both Eyes are Open OR Classifier Could Not detect Eyes
            eye_status +='1'
        else:
            
            # Devide Face into Two halves (Left and Right)
            left_face_color = frame[y:y+h, x+int(w/2):x+w]
            left_face_gray = grayImage[y:y+h, x+int(w/2):x+w]
            right_face_color = frame[y:y+h, x:x+int(w/2)]
            right_face_gray = grayImage[y:y+h, x:x+int(w/2)]
            
            '''plt.imshow(left_face_color)
            plt.title('Left Face')
            plt.show()
            
            plt.imshow(right_face_color)
            plt.title('Right Face')
            plt.show()'''
            #Detect Left and Right Eye
            left_eye = left_eye_cascade.detectMultiScale(left_face_gray)
            right_eye = right_eye_cascade.detectMultiScale(right_face_gray)
            
            
            if (len(left_eye) <1 | len(right_eye) < 1):
                #print('Eye Detection Failed ....')
                eye_status += '1'
            else:
                #Left Eye Status Check
                for lx,ly,lw,lh in left_eye:
                    left_eye_color = left_face_color[ly:ly+lh, lx:lx+lw]
                    #cv.rectangle(left_eye_color,(lx,ly),(lx+lw,ly+lw),(255,0,0),5)
            
                    '''plt.imshow(left_eye_color)
                    plt.title('Left Eye')
                    plt.show()'''
                    pred_class = predict(left_eye_color)
                    
                if pred_class == '1':
                    for rx,ry,rw,rh in right_eye:
                        right_eye_color = right_face_color[ry:ry+rh, rx:rx+rw]
                        #cv.rectangle(right_eye_color,(rx,ry),(rx+rw,ry+rw),(255,0,0),5)
                        '''plt.imshow(right_eye_color)
                        plt.title('Right Eye')
                        plt.show()'''
                        pred_class = predict(right_eye_color)
                
                eye_status +=pred_class
    
        #print('Eye Pattern ', eye_status)
        if re.search('10+|01+', eye_status):
            status = 'Real'
            #print('Status ', status)
            #break
        elif len(eye_status) == 20:
            status = 'Fake'
            #print('Status ', status)
        
        
        if status != '':
            cv.putText(frame, status, (x,y), cv.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 4)
            
        cv.imshow('Image',frame)
        
        
    frameCount +=1
    
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
# When everything done, release the capture
video.release()
cv.destroyAllWindows()
